# Исследование надёжности заёмщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита

**ПЛАН РАБОТЫ:**
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропущенных значений](#null)
    * [Замена типовых данных](#astype)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemmatize)
    * [Категоризация данных](#groupby)
3. [Вывод](#itog)



<a id="start"></a>

###  Открытие файла с данными и изучение общей информации. 

In [1]:
import pandas as pd
clients_base = pd.read_csv('/datasets/data.csv')


In [2]:
clients_base.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<a id="preprocessing"></a>
### Предобработка данных

<a id="null"></a>
#### Обработка пропусков

In [3]:
clients_base.info() #Оценим объём данных в таблице и есть ли NAN-ы.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски есть только в 2-х столбцах "days_employed"(), "total_income", и кол-во пропусков одинаковое.Скорее всего, эти пропуски взаимосвязаны и неслучайные. Видимо, сведения о доходах отсутствуют, потому что люди не работают. К этой категории можно отнести неработающих студентов, тех, кто находится в декретном отпуске, тип занятости -безработные.

In [4]:
#проверка взаимосвязи пропусков 2-х столбцов. Получилось 2174 одинаковых пропусков, а это и есть все пропуски.
clients_base[(clients_base['days_employed'].isnull()) & (clients_base['total_income'].isnull())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [5]:
#Так лучше видно:
clients_base.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
#Посмотрим уникальные значения по всем столбцам.
for row in clients_base:
    unik=clients_base[row].unique()
    print(row, unik)

children [ 1  0  3  2 -1  4 20  5]
days_employed [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
dob_years [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
education ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
education_id [0 1 2 3 4]
family_status ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
family_status_id [0 1 2 3 4]
gender ['F' 'M' 'XNA']
income_type ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
debt [0 1]
total_income [253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]
purpose ['по

Помимо пропусков, есть еще некорректные данные: кол-во детей -значения 20 и -1, СТАЖ - отрицательные значения и заметили тут нереальный стаж даже присутствует-порядка 930 лет, ОБРАЗОВАНИЕ- разные регистры, ПОЛ- "XNA"

In [7]:
#разберемся с кол-вом детей. Заменим значения 20 на 2, -1 на 1
clients_base.loc[clients_base['children'] == 20, 'children'] = 2 
clients_base.loc[clients_base['children'] == -1, 'children'] = 1
print("Кол-во детей", clients_base['children'].unique()) #проверим

Кол-во детей [1 0 3 2 4 5]


In [8]:
#Разберемся со значением "XNA" в столбце [gender]
#узнаем сперва их кол-во
clients_base_count_xna=clients_base.loc[clients_base['gender'] =='XNA']
print(clients_base_count_xna['gender'].count())

1


In [9]:
#Видим, что значение всего одно и поэтому его удаление или замена на другой пол не приведет к искажению наших данных.
# Да и решение поставленных вопросов никак не зависит от пола клиентов.
#Вставим сюда любой пол replace(), 
#т.е. меняем на "пропуск"
clients_base['gender']=clients_base['gender'].replace('XNA', 'F')
print(clients_base['gender'].unique())#  проверка на присутствие значения

['F' 'M']


In [10]:
# избавимся от отрицательного стажа с помощью вызова функции abs()
clients_base['days_employed'] = clients_base['days_employed'].abs()
clients_base['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

In [11]:
# необходимо сделать группировку по типам занятости и выделить среднее значение по стажу для каждого типа занятости 
#и вставить эти значения вместо нулей
#вводим переменную, которая нам дает среднее значение стажа группировки по типу занятости 

mean_itype_days_employed = clients_base.groupby(['income_type'])['days_employed'].mean()
#задаем функцию, которая на вход принимает строковое значение
def income_type_de(row):
    income_type = row['income_type'] 
    days_employed = row['days_employed']
#мы в этой строчке проверяем, что значение не является пропуском.
    if pd.isna(days_employed):
        row['days_employed'] = mean_itype_days_employed.loc[income_type]
    return row

clients_base = clients_base.apply(income_type_de, axis = 1)
print(clients_base.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [12]:
# необходимо сделать группировку по типам занятости и выделить медиану значения по доходу для каждого типа занятости 
#и вставить эти значения вместо нулей
#вводим переменную, которая нам дает среднее значение стажа группировки по типу занятости 
mean_itype_total_income  = clients_base.groupby(['income_type'])['total_income'].median()
#по алгоритму, указанному выше
def income_type_ti(row):
    income_type = row['income_type']
    total_income = row['total_income']
    
    if pd.isna(total_income):
        row['total_income'] = mean_itype_total_income.loc[income_type]
    return row

clients_base = clients_base.apply(income_type_ti, axis = 1)
print(clients_base.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Отлично! Избавились от всех пропусков. Нашли все "неадекватные" значения, заменили их адекватными, соответствующими значениями остальным значением столбца. Удалили все "NAN-ы". Данные откорректированы. Идем дальше...

<a id="astype"></a>
#### Замена типа данных

In [13]:
# изменим тип данных столбцов 2 и 11 с типа "float" на "int". применим способ через loc., сокращенную запись
clients_base[['days_employed','total_income']]=clients_base[['days_employed','total_income']].astype('int')
clients_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB



Для замены типа данных был использован метод 'astype', т.к. надо было тип указать 'int'. Метод to_numeric() используется для преобразования в тип 'float'.

<a id="duplicates"></a>
#### Обработка дубликатов

In [14]:
#Чтобы учесть правильно все дубликаты, все символы в строках приведем к нижнему регистру вызовом метода lower():
clients_base['education'] = clients_base['education'].str.lower()
clients_base['family_status'] = clients_base['family_status'].str.lower()
clients_base['gender'] = clients_base['gender'].str.lower()
clients_base['income_type'] = clients_base['income_type'].str.lower()
clients_base['purpose'] = clients_base['purpose'].str.lower()
print(clients_base['education'].unique())
print(clients_base['family_status'].unique())
print(clients_base['gender'].unique())
print(clients_base['income_type'].unique())
print(clients_base['purpose'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
['f' 'm']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего ж

In [15]:
#узнаем кол-во дубликатов
print(clients_base.duplicated().sum()) 

71


In [16]:
#удалим дубликаты
clients_base = clients_base.drop_duplicates()
clients_base.duplicated().sum() #проверка

0

#### ВЫВОД:
Файл мы почистили хорошо: обработали данные, установили корректные типы данных и удалили дубликаты. Дубликаты, вероятно, возникли по каким-то техническим причинам. Может быть, данные были получены путем слияния информации с разных источников, и поэтому получились повторы.

<a id="lemmatize"></a>
#### Лемматизация

In [17]:
# pymystem3 импортируется так:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [18]:
#рассмотрим весь перечень целей кредита
clients_base['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [19]:
# создаем список с главными словами
spisok = ["свадьба", "недвижимость", "жилье", "автомобиль", "образование"]
def lemmatize(row): #напишем функцию, которая на вход принимает строковое значение
    lemma = m.lemmatize(row) #внутри функции сделали лемматизацию этой строки, которая приходит на вход
    for word in spisok: 
        if word in lemma: #если слово в лемме входит, то возвращать (return) из функции конкретное значение категории
            lemma = word
    return lemma

#применить функцию с помощью apply к столбцу с целями кредита, а результат записывать 
# в новый столбец по которому планируется в дальнейшем группировка
clients_base['purpose_lemma'] = clients_base['purpose'].apply(lemmatize)        
clients_base.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба


In [20]:
# Посчитаем весь список целей кредита.
clients_base['purpose_lemma'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_lemma, dtype: int64

Добавили стобец, с помощью которого стало удобнее работать, получили удобный короткий список целей получения кредита.

1. По семейному статусу:

In [21]:
# Применим метод value_counts() к столбцу 'family_status' для подсчёта количества сем.положений.
print(clients_base['family_status'].value_counts())

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [22]:
# Cколько сем.положений всего:
print(len(clients_base['family_status'].unique())) 

5


In [23]:
# Создадим словарь и оптимизируем данные.
family_dict = clients_base[['family_status','family_status_id']]
print(family_dict.head(10)) 

      family_status  family_status_id
0   женат / замужем                 0
1   женат / замужем                 0
2   женат / замужем                 0
3   женат / замужем                 0
4  гражданский брак                 1
5  гражданский брак                 1
6   женат / замужем                 0
7   женат / замужем                 0
8  гражданский брак                 1
9   женат / замужем                 0


In [24]:
#В «словаре» заметили большое количество дубликатов. Их нужно удалить.
#Применим цепочку методов: drop_duplicates() и reset_index().
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
print(family_dict) 

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  не женат / не замужем                 4


2. Словарь по образованию:

In [25]:
# Применим метод value_counts() к столбцу 'education_id' для подсчёта количества типов образований  
print(clients_base['education'].value_counts())

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [26]:
# Cколько id-образований всего:
print(len(clients_base['education'].unique()))

5


In [27]:
# Создадим словарь и оптимизируем данные.
education_dict = clients_base[['education','education_id']]
print(education_dict.head(10)) 

  education  education_id
0    высшее             0
1   среднее             1
2   среднее             1
3   среднее             1
4   среднее             1
5    высшее             0
6    высшее             0
7   среднее             1
8    высшее             0
9   среднее             1


In [28]:
#В «словаре» заметили большое количество дубликатов. Их нужно удалить.
#Применим цепочку методов: drop_duplicates() и reset_index().
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict) 

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


<a id="groupby"></a>
#### Категоризация данных

##### Зависимость сведений о кол-ве детей и возвратом кредита в срок

In [29]:
# Поработаем с качественными данными, а именно со сведениями о кол-ве детей у клиентов банка
clients_base['children'].unique() #вспомним об уникальных значениях

array([1, 0, 3, 2, 4, 5])

In [30]:
#сделаем группировку по кол-ву детей. Увидим зависимость задолженностей от кол-ва детей.
#Вызовем метод agg(), указывающий, какие именно функции применить к ['children']
clients_base_child=clients_base.groupby('children').agg({'debt': ['count', 'sum','mean']})
print(clients_base_child.sort_values([('debt', 'mean')],ascending=True)) #сортируем по убыванию по сред.значению

           debt                
          count   sum      mean
children                       
5             9     0  0.000000
0         14091  1063  0.075438
3           330    27  0.081818
1          4855   445  0.091658
2          2128   202  0.094925
4            41     4  0.097561


Видим прямую зависимость, что чем больше детей, тем труднее клиентам справляться с долгами. Только клиенты с кол-вом "5" детей не имеют вообще задолженностей. Посчитаем сколько вообще таких клиентов.

In [31]:
print((clients_base['children'] == 0).sum())
print((clients_base['children'] == 1).sum())
print((clients_base['children'] == 2).sum())
print((clients_base['children'] == 3).sum())
print((clients_base['children'] == 4).sum())
print((clients_base['children'] == 5).sum())

14091
4855
2128
330
41
9


Видим что кол-во клиентов с 5-мя детьми 9 человек, что не дает такой важной информации и не портит нашу статистику. Поэтому вывод остается прежним.


##### Зависимость между семейным положением и возвратом кредита в срок.

In [32]:
#рассмотрим весь перечень столбца о семейном положении и посчитаем
clients_base['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [33]:
#сделаем группировку по сем.положению. Увидим зависимость задолженностей от кол-ва детей
clients_base_family_status_debt=clients_base.groupby('family_status').agg({'debt': ['count', 'sum','mean']})
print(clients_base_family_status_debt.sort_values([('debt', 'mean')],ascending=True))

                        debt               
                       count  sum      mean
family_status                              
вдовец / вдова           959   63  0.065693
в разводе               1195   85  0.071130
женат / замужем        12339  931  0.075452
гражданский брак        4151  388  0.093471
не женат / не замужем   2810  274  0.097509



Чаще всего просрочки наблюдаются у "свободных птиц", за ними следуют уже люди, находящиеся в гражданском браке, и самые надежные это клиенты - вдовцы.

##### Зависимость между уровнем дохода и возвратом кредита в срок 

Тут метод, используемый выше, не подойдет, т.к. уникальных значений множество. Логичнее будет разбить сведения о доходах на несколько групп.
Используем квантили 25%, 50% и 75%

In [34]:
statik=clients_base['total_income'].describe()
statik[4:7]

25%    107623.00
50%    142594.00
75%    195820.25
Name: total_income, dtype: float64

In [35]:
#введем функцию для определения доходов по категориям
def total_income_group(income):
    if income<=statik[4]: 
        return "низкий" #до 107623.00
    if statik[4]<income<=statik[5]: 
        return "средний" # до 142594.00
    if statik[5]<income<=statik[6]:
        return "высокий" # до 195820.25
    return "выше среднего" # более 195820.25
# добавим столбец ['total_income_group']
clients_base['total_income_group'] = clients_base['total_income'].apply(total_income_group)
clients_base.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,жилье,выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,жилье,высокий
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,жилье,выше среднего
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,жилье,выше среднего
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,жилье,высокий


In [36]:
#Применим метод value_counts() к столбцу 'education_id' для подсчёта количества категорий по доходам
clients_base['total_income_group'].value_counts()

средний          5479
низкий           5364
выше среднего    5364
высокий          5247
Name: total_income_group, dtype: int64

In [37]:
#Сделаем сводную таблицу методом pivot_table()
clients_base_pivot = clients_base.pivot_table(index = ['total_income_group'], columns = 'debt', values = 'gender', aggfunc = 'count')
clients_base_pivot

debt,0,1
total_income_group,,
высокий,4799,448
выше среднего,4981,383
низкий,4937,427
средний,4996,483


In [38]:
# найдем вероятность задолженности для каждой группы доходов
clients_base_pivot['ratio %'] = (round(clients_base_pivot[1] / (clients_base_pivot[0] + clients_base_pivot[1]), 3))*100
clients_base_pivot.sort_values('ratio %', ascending = False)

debt,0,1,ratio %
total_income_group,,,
средний,4996,483,8.8
высокий,4799,448,8.5
низкий,4937,427,8.0
выше среднего,4981,383,7.1


##### Вывод
Самые надежные платильщики это клиенты с доходом выше среднего уровня, после них можно поверить клиентам с низким уровнем дохода.

#### Как разные цели кредита влияют на его возврат в срок?

In [39]:
#создаем таблицу data_pivot_income где смотрим должников по целям кредита
clients_base_pivot_purpose = clients_base.pivot_table(index='purpose_lemma', columns='debt', values='purpose', aggfunc='count')
clients_base_pivot_purpose

debt,0,1
purpose_lemma,,
автомобиль,3903,403
жилье,4152,308
недвижимость,5877,474
образование,3643,370
свадьба,2138,186


In [40]:
#считаем процент должников
clients_base_pivot_purpose['ratio %'] = (round(clients_base_pivot_purpose[1] / (clients_base_pivot_purpose[1]+clients_base_pivot_purpose[0]),3)) * 100

print(clients_base_pivot_purpose.sort_values(['ratio %'],ascending=True))

debt              0    1  ratio %
purpose_lemma                    
жилье          4152  308      6.9
недвижимость   5877  474      7.5
свадьба        2138  186      8.0
образование    3643  370      9.2
автомобиль     3903  403      9.4


###### Вывод:  
Заемщики, берущие кредит для операций, связанных с жильем, наиболее ответственны.

<a id="itog"></a>
### Общий вывод

1.Кол-во детей влияет на способность клиентов оплачивать свои долги перед банком в срок. Конечно, дети -это цветы жизни, которые нужно поливать и растить, поэтому это, конечно, затраты и порой клиенты не всегда могут располагать своим финансовым положением.

2.Чаще всего просрочки наблюдаются у необременных браком клиентов, за ними следуют уже люди, находящиеся в браке "без штампа", и самые надежные это клиенты, пережившие потерю своих супругов.

3.Какая же у нас ситуация с зависимостью между уровнем дохода и возвратом кредита в срок? Тут...Самые надежные платильщики это клиенты с доходом выше среднего уровня, после них, мы нисколько не удивлены, что можно верить людям с меньшим доходом: они-то могут адекватно распределять свой доход.

4.Как разные цели кредита влияют на его возврат в срок? - Жильё приобретать/проводить операции с ним - это ответсвенное дело и чаще всего на очень длительное время, поэтому подход к нему более серьезный. Да и процент всегда меньше, чем у потребительского кредита. Люди, приобретающие автомобиль и "гранит науки" из данного перечня менее ответственные.